Test out some ways of interacting with Renshuu API.
-Estimate proficiency of user.
-Select terms to use in conversation.
-Get examples etc.

In [1]:
import os
import json
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import requests

In [3]:
RENSHUU_API_KEY = os.getenv('RENSHUU_API_KEY')

In [5]:
HEADERS = {
    'Authorization': f'Bearer {RENSHUU_API_KEY}',
    'Content-Type': 'application/json'
}

In [6]:
# use these keywords to identify the level of a schedule
SCHEDULE_KEYWORDS = [
    (
        "n5",
        ["n5", "beginner"]
    ),
    (
        "n4",
        ["n4", "pre-intermediate", "preintermediate"]
    ),
    (
        "n3",
        ["n3", "intermediate"]
    ),
    (
        "n2",
        ["n2", "pre-advanced", "preadvanced"]
    ),
    (
        "n1",
        ["n1", "advanced"]
    ),
]
LEVELS = ["n5", "n4", "n3", "n2", "n1"]

In [9]:
def estimate_proficiency_from_profile(threshold = 0.5):
    '''
    Estimate the users proficiency based on his profile level progress
    Will get the highest level with atleast <threshold> progress
    '''
    profile = requests.get(
        "https://api.renshuu.org/v1/profile", 
        headers=HEADERS
    ).json()

    level_proficiencies = {}
    for level in LEVELS:
        # assume equal importance of sentence, grammar, kanji and vocab
        level_proficiencies[level] = (\
            profile["level_progress_percs"]["vocab"][level]\
            + profile["level_progress_percs"]["kanji"][level]\
            + profile["level_progress_percs"]["grammar"][level]\
            + profile["level_progress_percs"]["sent"][level]\
        )/400

    estimated_level = "n5"
    for level in LEVELS[1:]:
        if level_proficiencies[level] >= threshold:
            estimated_level = level

    return {
        "level_proficiencies": level_proficiencies,
        "estimated_level": estimated_level,
    }

proficiency = estimate_proficiency_from_profile()
print(proficiency)

{'level_proficiencies': {'n5': 1.0, 'n4': 0.98, 'n3': 0.0475, 'n2': 0.0025, 'n1': 0.0}, 'estimated_level': 'n4'}
